<a href="https://colab.research.google.com/github/Gustavolorenzz/IA/blob/main/Breast_cancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImPipeline

# Lendo dados do arquivo CSV.
df = pd.read_csv('/content/breast-cancer.csv')

# Ajustando os dados: Transformação de texto em números e ajuste de escala.
num_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])
cat_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])
preprocess = ColumnTransformer([
    ('num', num_pipe, ['deg-malig']),
    ('cat', cat_pipe, ['age', 'menopause', 'tumor-size', 'inv-nodes', 'node-caps', 'breast', 'breast-quad', 'irradiat'])
])

# Separando as variáveis independentes da dependente.
X, y = df.drop('Class', axis=1), df['Class'].map({'no-recurrence-events': 0, 'recurrence-events': 1})

# Dividindo o conjunto de dados para treino e teste.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

# Aplicando o pré-processamento e balanceando as classes no treino.
X_train_prep = preprocess.fit_transform(X_train)
X_test_prep = preprocess.transform(X_test)
X_train_bal, y_train_bal = SMOTE().fit_resample(X_train_prep, y_train)

# Definição da pipeline completa com SMOTE e MLPClassifier.
pipeline = ImPipeline([
    ('preprocessor', preprocess),
    ('smote', SMOTE()),
    ('classifier', MLPClassifier(max_iter=1000, verbose=False))
])

# Parâmetros para o Grid Search com objetivo de otimizar a rede neural.
params_grid = {
    'classifier__hidden_layer_sizes': [(10,), (10, 10), (20, 10)],
    'classifier__learning_rate_init': [0.001, 0.01, 0.1, 0.2, 0.3],
    'classifier__max_iter': [1000],
    'classifier__verbose': [False],
    'classifier__n_iter_no_change': [10],
    'classifier__tol': [0.0001]
}

# Implementação do Grid Search com Cross-Validation.
grid_search_cv = GridSearchCV(
    pipeline, params_grid, cv=5, scoring='accuracy',
    verbose=0, n_jobs=1, error_score='raise'
)

# Execução do Grid Search no conjunto de treino.
grid_search_cv.fit(X_train, y_train)

# Selecionando o melhor modelo encontrado.
optimal_model = grid_search_cv.best_estimator_

# Avaliação do modelo ótimo no conjunto de teste.
predictions = optimal_model.predict(X_test)

# Apresentação dos resultados de desempenho.
print("Relatório de Desempenho do Modelo Ótimo:")
print(f"Acurácia: {accuracy_score(y_test, predictions):.2f}")
print("Matriz de Confusão:")
print(confusion_matrix(y_test, predictions))
print("\nResumo de Classificação:")
print(classification_report(y_test, predictions))

# Exibindo os parâmetros ótimos.
print("Configurações Ótimas:")
print(grid_search_cv.best_params_)
print(f"Precisão Média da Validação Cruzada: {grid_search_cv.best_score_:.2f}")

# Repetindo o processo para diferentes topologias e taxas de aprendizado.
topology_results = []
learning_rate_results = []

for size in [(10,), (10, 10), (20, 10)]:
    for lr in [0.001, 0.01, 0.1, 0.2, 0.3]:
        model = MLPClassifier(hidden_layer_sizes=size, learning_rate_init=lr, max_iter=1000, verbose=False)
        model.fit(X_train_bal, y_train_bal)

        # Predições e avaliação para cada configuração
        pred = model.predict(X_test_prep)
        accuracy = accuracy_score(y_test, pred)
        conf_mat = confusion_matrix(y_test, pred)
        class_rep = classification_report(y_test, pred, output_dict=True)

        # Armazenando os resultados
        topology_results.append((size, lr, model.n_iter_, accuracy))
        learning_rate_results.append((lr, model.n_iter_, accuracy))

# Apresentando os resultados para diferentes topologias e taxas de aprendizado.
print("\nResultados para Diferentes Topologias e Taxas de Aprendizado:")
for res in topology_results:
    size, lr, epochs, accuracy = res
    print(f"Topologia: {size}, Taxa de Aprendizado: {lr}, Épocas: {epochs}, Acurácia: {accuracy:.2f}")

print("\nResultados para Diferentes Taxas de Aprendizado:")
for res in learning_rate_results:
    lr, epochs, accuracy = res
    print(f"Taxa de Aprendizado: {lr}, Épocas: {epochs}, Acurácia: {accuracy:.2f}")

# Resumo final usando o melhor modelo do Grid Search.
print("\nResumo Final Usando o Melhor Modelo do Grid Search:")
print(f"Melhor Configuração de Hiperparâmetros: {grid_search_cv.best_params_}")
print(f"Precisão Média da Validação Cruzada: {grid_search_cv.best_score_:.2f}")
print(f"Acurácia no Conjunto de Teste: {accuracy_score(y_test, predictions):.2f}")
print("Matriz de Confusão:")
print(confusion_matrix(y_test, predictions))
print("Relatório de Classificação:")
print(classification_report(y_test, predictions))




/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perce

Relatório de Desempenho do Modelo Ótimo:
Acurácia: 0.71
Matriz de Confusão:
[[31 10]
 [ 7 10]]

Resumo de Classificação:
              precision    recall  f1-score   support

           0       0.82      0.76      0.78        41
           1       0.50      0.59      0.54        17

    accuracy                           0.71        58
   macro avg       0.66      0.67      0.66        58
weighted avg       0.72      0.71      0.71        58

Configurações Ótimas:
{'classifier__hidden_layer_sizes': (20, 10), 'classifier__learning_rate_init': 0.01, 'classifier__max_iter': 1000, 'classifier__n_iter_no_change': 10, 'classifier__tol': 0.0001, 'classifier__verbose': False}
Precisão Média da Validação Cruzada: 0.67


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` paramet


Resultados para Diferentes Topologias e Taxas de Aprendizado:
Topologia: (10,), Taxa de Aprendizado: 0.001, Épocas: 1000, Acurácia: 0.67
Topologia: (10,), Taxa de Aprendizado: 0.01, Épocas: 480, Acurácia: 0.78
Topologia: (10,), Taxa de Aprendizado: 0.1, Épocas: 112, Acurácia: 0.72
Topologia: (10,), Taxa de Aprendizado: 0.2, Épocas: 120, Acurácia: 0.67
Topologia: (10,), Taxa de Aprendizado: 0.3, Épocas: 80, Acurácia: 0.66
Topologia: (10, 10), Taxa de Aprendizado: 0.001, Épocas: 1000, Acurácia: 0.62
Topologia: (10, 10), Taxa de Aprendizado: 0.01, Épocas: 219, Acurácia: 0.66
Topologia: (10, 10), Taxa de Aprendizado: 0.1, Épocas: 112, Acurácia: 0.57
Topologia: (10, 10), Taxa de Aprendizado: 0.2, Épocas: 74, Acurácia: 0.69
Topologia: (10, 10), Taxa de Aprendizado: 0.3, Épocas: 20, Acurácia: 0.29
Topologia: (20, 10), Taxa de Aprendizado: 0.001, Épocas: 708, Acurácia: 0.71
Topologia: (20, 10), Taxa de Aprendizado: 0.01, Épocas: 131, Acurácia: 0.66
Topologia: (20, 10), Taxa de Aprendizado: 0.